##Results for SMAP Signals (Tabel II and IV)

In [5]:
import io
import os
import random
import urllib.request
import zipfile
import datetime
import pdb
import holidays
import json
import copy
import time
import numpy as np
import torch
import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from torch.profiler import profile, record_function, ProfilerActivity
from sklearn.model_selection import train_test_split

train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')
seed=42
torch.backends.cudnn.determinestic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


def build_df(data, start=0):
#
    index = np.array(range(start, start + len(data)))
    timestamp = index * 86400 + 1022819200



    return pd.DataFrame({'timestamp': timestamp.astype(int), 'value': data[:, 0], 'index': index.astype(int)})

Calculations = pd.DataFrame()
def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)


def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))
class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):
        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size  # user-defined

        self.hidden_size_1 = 128
        self.hidden_size_2 = 32
        self.stacked_layers = 2
        self.dropout_probability = 0.2  # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True, dropout=self.dropout_probability)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True,dropout=self.dropout_probability)

        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)

        output, _ = self.lstm1(x, hidden)

        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :]  # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']





SMAP=[  'P-1', 'S-1', 'E-1', 'E-2','E-5', 'E-6', 'E-7',
    'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13',   'P-3',
     'A-2', 'A-4', 'G-2',
    'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9',
    'G-4', 'T-3',  'D-12', 'B-1',  'G-7', 'P-7',
     'A-5', 'A-7', 'D-13',  'A-9']

MSL=[
    'M-1', 'S-2',  'T-4', 'T-5', 'F-7',  'M-4',
    'M-5',  'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9',  'T-8',  'D-15', 'M-7', 'F-8']

#SMAP=pd.Series(SMAP)
#MSL=pd.Series(MSL)


avg=[]
train_signals=SMAP
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()
MSE=[]
df_label=df_label[df_label['chan_id'].isin(SMAP)]

df_label=df_label.reset_index(drop=True)
TrainingTime=[]
TestingTime=[]
NumberOfParameters=[]
for name in SMAP:
        label_row=df_label[df_label.chan_id==name]
       # pdb.set_trace()
     
        labels= label_row.anomaly_sequences[im]
        labels=eval(labels)
      
        true_indices_grouped = [list(range(e[0], e[1]+1)) for e in labels]
        true_indices_flat = set([i for group in true_indices_grouped for i in group])


        appended_data = []

    
        labels= true_indices_flat
       

       # for i in range(len(labels)):
        #    anom=labels[i]
            #start=anom[0]
            #end=anom[1]

        index = list(labels)

       # timestamp = index * 86400 + 1022819200


        anomalies=pd.DataFrame({'value': 1, 'index':index})
       #     appended_data.append(anomalies)

        label_data = anomalies

        #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
        #label_data ['month'] = label_data ['date'].dt.month.astype(int)
        label_data ['name'] = name
        #label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
        #label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
        label_data  = label_data [selected_columns]
        #label_data .to_csv('csv/' + name + '.csv', index=False)


        signal=name
        train_np = np.load('data/train/' + signal + '.npy')
        test_np = np.load('data/test/' + signal + '.npy')


        data = build_df(np.concatenate([train_np, test_np]))
        data['date'] = pd.to_datetime(data['timestamp'], unit='s')
        data['month'] = data['date'].dt.month.astype(int)
        data['name'] = name
        data['index'] = data['index'].astype(int)
        data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
        data['hour_of_day'] = data['date'].dt.hour.astype(int)
        data = data[selected_columns]
        _ , validation = train_test_split(
            data,
            test_size=0.15
        )

        val_dataset=validation

        val=val_dataset


        #val['index'] = val['index'].astype(int)
        #val['date'] = pd.to_datetime(val['timestamp'], unit='s')
        #val['month'] = val['date'].dt.month.astype(int)
        #val['day_of_month'] = val['date'].dt.day.astype(int)
        #val['name'] = name
        #val['day_of_week'] = val['date'].dt.dayofweek.astype(int)
        #val['hour_of_day'] = val['date'].dt.hour.astype(int)
        #val['index'] = val['index'].astype(int)
        #val = val[selected_columns]

        train = build_df(train_np)
        train['date'] = pd.to_datetime(train['timestamp'], unit='s')
        train['month'] = train['date'].dt.month.astype(int)
        train['day_of_month'] = train['date'].dt.day.astype(int)
        train['name'] = name
        train['day_of_week'] = train['date'].dt.dayofweek.astype(int)
        train['hour_of_day'] = train['date'].dt.hour.astype(int)
        train['index'] = train['index'].astype(int)
        train = train[selected_columns]
        # train.to_csv('csv/' + name + '.csv', index=False)
        # train.to_csv('csv/' + name + '-train.csv', index=False)


        test = build_df(test_np, start=len(train))
        test['date'] = pd.to_datetime(test['timestamp'], unit='s')
        test['month'] = test['date'].dt.month.astype(int)
        test['name'] = name
        test['day_of_week'] = test['date'].dt.dayofweek.astype(int)
        test['hour_of_day'] = test['date'].dt.hour.astype(int)
        test['index'] = test['index'].astype(int)
        test = test[selected_columns]
        # test.to_csv('csv/' + name + '.csv', index=False)
        # test.to_csv('csv/' + name + '-train.csv', index=False)
        # test.to_csv('csv/' + name + '-test.csv', index=False)

        datetime_columns = ['index',  'name']
        target_column = 'value'

        feature_columns = datetime_columns + ['value']

        resample_df = train[feature_columns]
        resample_df_test = test[feature_columns]
        resample_df_val=val[feature_columns]
        #print(resample_df)

       # plot_length = 1000000
        #plot_df = resample_df.copy(deep=True).iloc[:plot_length]
        #plot_df['weekday'] = plot_df['date'].dt.day_name()

        n_train = len(train_np)
        n_test = len(test_np)

        features = ['index',  'value']
        feature_array = resample_df[features].values
        feature_array_test = resample_df_test[features].values
        feature_array_val = resample_df_val[features].values
        # Fit Scaler only on Training features
        feature_scaler = MinMaxScaler()
        feature_scaler.fit(feature_array[:n_train])
        # Fit Scaler only on Training target values
        #feature_scaler.fit(feature_array_test[:n_test])

        target_scaler = MinMaxScaler()
        target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

        # Transfom on both Training and Test data
        scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                                    columns=features)

        scaled_array_test = pd.DataFrame(feature_scaler.transform(feature_array_test),
                                         columns=features)
        scaled_array_val = pd.DataFrame(feature_scaler.transform(feature_array_val),
                                         columns=features)

        sequence_length = 10

        X_train, y_train = create_sliding_window(scaled_array,
                                                 sequence_length)

        X_test, y_test = create_sliding_window(scaled_array_test,
                                               sequence_length)
        X_val, y_val = create_sliding_window(scaled_array_val,
                                               sequence_length)

        #sns.displot(data=y_test,  kind="hist", bins=100, aspect=1.5)
      #  plt.show()
       # pdb.set_trace()

        #f = Fitter(y_train,
         #          distributions=['gamma',
          #                        'lognorm',
           #                       "beta",
            #                      "burr",
             #                     "norm",'cauchy', 'chi2', 'expon', 'exponpow', 'powerlaw', 'rayleigh', 'uniform'])
        #f.fit()
        #f.summary()
        #pdb.set_trace()
        n_features = scaled_array.shape[-1]
        sequence_length = 10
        output_length = 1

        batch_size = 64
        n_epochs = 10
        learning_rate = 0.01

        bayesian_lstm = BayesianLSTM(n_features=n_features,
                                     output_length=output_length,
                                     batch_size=batch_size)
        

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
        bayesian_lstm.train()
        start=time.time()

        for e in range(1, n_epochs + 1):
            for b in range(0, len(X_train), batch_size):
                features = X_train[b:b + batch_size, :, :]
                target = y_train[b:b + batch_size]

                X_batch = torch.tensor(features, dtype=torch.float32)
                y_batch = torch.tensor(target, dtype=torch.float32)
               # pdb.set_trace()

                output = bayesian_lstm(X_batch)
                loss = criterion(output.view(-1), y_batch)


                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if e % 10 == 0:
                print('epoch', e, 'loss: ', loss.item())
                offset = sequence_length

        training_df = pd.DataFrame()
       # training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1]
        training_df['index'] = resample_df['index'].iloc[offset:n_train + offset:1]
        training_predictions = bayesian_lstm.predict(X_train)
        end=time.time()
        NumberOfTrainableParameters=sum(p.numel() for p in bayesian_lstm.parameters() if p.requires_grad)
        TrainingingTime=(end-start)/n_epochs
        NumberOfParameters.append(NumberOfTrainableParameters)
        TrainingTime.append(TrainingingTime)
       



        print("Training time:",(end-start)/n_epochs,'Signal', name)
    
        print(" Number Of Trainable Parameters", NumberOfTrainableParameters,'Signal', name)
   


        training_df['value'] = inverse_transform(training_predictions)
        training_df['source'] = 'Training Prediction'


        #training_truth_df['date'] = training_df['date']
        training_truth_df['index'] = training_df['index']
        training_truth_df['value'] = resample_df['value'].iloc[
                                     offset:n_train + offset:1]
        training_truth_df['source'] = 'True Values'

        testing_df = pd.DataFrame()
       # testing_df['date'] = resample_df_test['date'].iloc[offset:n_test + offset:1]
        testing_df['index'] = resample_df_test['index'].iloc[offset:n_test + offset:1]
        start=time.time()

        testing_predictions = bayesian_lstm.predict(X_test)
        end=time.time()
        TestingingTime=(end-start)/n_epochs
        print(" Testing Time", (end-start)/n_epochs,'Signal', name)
        TestingTime.append(TestingingTime)

        #print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


        
                     
      




        testing_df['value'] = inverse_transform(testing_predictions)
        testing_df['source'] = 'Test Prediction'

        testing_truth_df = pd.DataFrame()
        #testing_truth_df['date'] = testing_df['date']
        testing_truth_df['index'] = testing_df['index']

        testing_truth_df['value'] = resample_df_test['value'].iloc[offset:n_test + offset:1]
        testing_truth_df['source'] = 'True Values'


        evaluation = pd.concat([training_df,
                                testing_df,
                                training_truth_df,
                                testing_truth_df
                                ], axis=0)

        fig = px.line(evaluation,
                             x="index",
                              y="value",
                              color="source")
        #fig.show()

        n_experiments = 2
        test_uncertainty_df = pd.DataFrame()
        #test_uncertainty_df['date'] = testing_df['date']
        test_uncertainty_df['index'] = testing_df['index']

        for i in range(n_experiments):
            experiment_predictions = bayesian_lstm.predict(X_test)

            test_uncertainty_df['value_{}'.format(i)] = inverse_transform(experiment_predictions)

        log_energy_consumption_df = test_uncertainty_df.filter(like='value', axis=1)
        test_uncertainty_df['value_mean'] = log_energy_consumption_df.mean(axis=1)
        test_uncertainty_df['value_std'] = log_energy_consumption_df.std(axis=1)
        MSE.append(mean( testing_truth_df['value']-test_uncertainty_df['value_mean'])**2)
        print(MSE)

 


        test_uncertainty1 = test_uncertainty_df['value_mean']

        #########################Adaptive Smoothing

        #   pdb.set_trace()
        Threshold = 10
        gamma = 0.1
        betta = 0.1
        test_uncertainty2 = []
        ###########
        for jj in range(len(test_uncertainty1) - 2):
            test_uncertaintyy = (1 / (1 + gamma + betta)) * (
                        test_uncertainty1.iloc[jj] + gamma * test_uncertainty1.iloc[jj + 1] + betta *
                        test_uncertainty1.iloc[jj + 2])
            test_uncertainty2.append(test_uncertaintyy)
        test_uncertainty2 = pd.DataFrame(test_uncertainty2)

        test_uncertainty2.columns = ["value_mean"]


        test_uncertainty_df["value_mean"] = test_uncertainty2["value_mean"]

        ########################################################


        test_uncertainty_df = test_uncertainty_df[['index',  'value_mean', 'value_std']]
        test_uncertainty_df['lower_bound'] = test_uncertainty_df['value_mean'] - 5* test_uncertainty_df['value_std']
        test_uncertainty_df['upper_bound'] = test_uncertainty_df['value_mean'] + 5 * test_uncertainty_df['value_std']
        import plotly.graph_objects as go

        test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
        # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
        truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
        # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

        upper_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['upper_bound'],
          mode='lines',
         fill=None,
         name='99% Upper Confidence Bound'
         )
        lower_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['lower_bound'],
          mode='lines',
          fill='tonexty',
         name= '99% Lower Confidence Bound',
         fillcolor='rgba(255, 211, 0, 0.1)',
         )
        real_trace = go.Scatter(
            x=truth_uncertainty_plot_df['index'],
            y=truth_uncertainty_plot_df['value'],
            mode='lines',
            fill=None,
            name='Real Values'
        )

        labels = go.Scatter(
           x=label_data['index'],
          y=label_data['value'],
         mode='lines',
         fill='tonexty' ,
         name='labels'
         )
        data = [upper_trace, lower_trace, real_trace]

        fig = go.Figure(data=data)
        fig.update_layout(title='Uncertainty MCDropout Test Data',
                            xaxis_title='index',
                            yaxis_title='value',
         legend_font_size=14,
                          )
        #fig.show()
        bounds_df = pd.DataFrame()

        # Using 99% confidence bounds
        bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
        bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
        bounds_df['real_value'] = truth_uncertainty_plot_df['value']
        bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

        bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                                  (bounds_df['real_value'] <= bounds_df['upper_bound']))

        print("Proportion of points contained within 99% confidence interval:",
              bounds_df['contained'].mean())
        predictedanomaly = bounds_df.index[~bounds_df['contained']]

        N = 15
        newarr = []


        for i in range(len(predictedanomaly) - N):
            if (predictedanomaly[i] + 1 == predictedanomaly[i + 1] and predictedanomaly[i + 1] + 1 == predictedanomaly[
               i + 2] and predictedanomaly[i+3]+1==predictedanomaly[i+4] and predictedanomaly[i+4]+1==predictedanomaly[i+5]
                and  predictedanomaly[i+5]+1==predictedanomaly[i+6] and predictedanomaly[i+6]+1==predictedanomaly[i+7]
                and predictedanomaly[i+7]+1==predictedanomaly[i+8]
                and predictedanomaly[i+8]+1==predictedanomaly[i+9]
                and predictedanomaly[i+9]+1==predictedanomaly[i+10]
                and predictedanomaly[i+10]+1==predictedanomaly[i+11]
                and predictedanomaly[i+11]+1==predictedanomaly[i+12]
                and predictedanomaly[i + 12] + 1 == predictedanomaly[i + 13]
                and predictedanomaly[i + 13] + 1 == predictedanomaly[i + 14]
                and predictedanomaly[i + 14] + 1 == predictedanomaly[i + 15]):

                newarr.append(predictedanomaly[i])
        #        newarr.append(predictedanomaly[i + 1])
         #       newarr.append(predictedanomaly[i + 2])


        predicteddanomaly = list(set(newarr))




        realanomaly = label_data['index']

        predicter=list(range(len(test_uncertainty2)))

        a1 = pd.DataFrame(index=range(len(test_uncertainty2)),columns=range(2))
        a1.columns=['index','value']

        a2 = pd.DataFrame(index=range(len(test_uncertainty2)), columns=range(2))
        a2.columns = ['index','value']




        for i in range(len(predicter)):
            if i in predicteddanomaly:
                a1.iloc[i,1]=1
            else:
                a1.iloc[i,1]=0

        for i in range(len(predicter)):
            if i in realanomaly:
                a2.iloc[i,1] = 1
            else:
                a2.iloc[i, 1] = 0



        y_real=a2.value
        y_real=y_real.astype(int)
        y_predi=a1.value
        y_predi=y_predi.astype(int)


        cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
        cm_plot_labels = ['no_anomaly', 'had_anomaly']
        plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

        # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
        #fp = len(predicteddanomaly) - tp
        #fn = 0
        #tn = len(truth_uncertainty_plot_df) - tp - fp - fn

        tp=cm[0][0]
        fp=cm[0][1]
        fn=cm[1][0]
        tn=cm[1][1]

        precision1 = tp / (tp + fp)
        recall1 = tp / (tp + fn)
        Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
        F11 = 2 / ((1 / precision1) + (1 / recall1))
        print('precision', precision1, 'Signal', name)
        print('recall', recall1, 'Signal', name)
        print('Accuracy', Accuracy1, 'Signal', name)
        print('F1', F11, 'Signal', name)
        precision.append(precision1)
        F1.append(F11)
        Accuracy.append(Accuracy1)
        recall.append(recall1)

        im=im+1



        #matched_indices = list(i_anom_predicted & true_indices_flat)

Calculations['pipline']='Bayesian_LSTM'
Calculations['fit_time']=TrainingTime
Calculations['prediction_time']=TestingTime

#Calculations.iloc['number_of_parameters']=NumberOfTrainableParameters
recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
#cm = confusion_matrix(y_true=test_labels, y_pred=predicteddanomaly)

################################################################################

# %%
recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
MSE_final=mean(MSE)
Accuracy_final=mean(Accuracy)
print('precision_final', precision_final)
print('recall_final', recall_final)
print('Accuracy_final',Accuracy_final)
print('F1_final', F1_final)
print('MSE_final', MSE_final)
## New Test Bayesian LSTM AE



epoch 10 loss:  0.009106989949941635
Training time: 3.108823299407959 Signal P-1
 Number Of Trainable Parameters 228897 Signal P-1
 Testing Time 0.26203978061676025 Signal P-1
[0.0015538817810600044]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:506: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.368452030606239
precision 0.9908292430896409 Signal P-1
recall 0.9124539074580706 Signal P-1
Accuracy 0.9047675103001765 Signal P-1
F1 0.9500278655025078 Signal P-1
epoch 10 loss:  0.0853353887796402
Training time: 1.793726348876953 Signal S-1
 Number Of Trainable Parameters 228897 Signal S-1
 Testing Time 0.15091581344604493 Signal S-1
[0.0015538817810600044, 1.504294963669963e-06]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:506: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.18289851113235897
precision 0.9318876437199826 Signal S-1
recall 0.9391317101789382 Signal S-1
Accuracy 0.8791148750170742 Signal S-1
F1 0.9354956534443714 Signal S-1
epoch 10 loss:  0.05176026374101639
Training time: 1.7407019138336182 Signal E-1
 Number Of Trainable Parameters 228897 Signal E-1
 Testing Time 0.18792994022369386 Signal E-1
[0.0015538817810600044, 1.504294963669963e-06, 0.016425899428125414]


...

Proportion of points contained within 99% confidence interval: 0.43892819858485094
precision 0.9921196526214217 Signal A-7
recall 0.7205933886228244 Signal A-7
Accuracy 0.7166222016007424 Signal A-7
F1 0.8348332092834426 Signal A-7
epoch 10 loss:  7.584533818771888e-07
Training time: 1.1250499963760376 Signal D-13
 Number Of Trainable Parameters 228897 Signal D-13
 Testing Time 0.20531878471374512 Signal D-13
[0.0015538817810600044, 1.504294963669963e-06, 0.016425899428125414, 0.36716248826688885, 0.012948204705798947, 8.87386111541333e-07, 0.00022666060227981947, 0.001966362049421288, 0.018527862588044577, 0.002562994389914008, 0.008975656751227901, 0.0004982309522393031, 0.0026710833156142343, 0.00011056170489142075, 0.000802129676171351, 0.000540648551107249, 2.879360355240949e-06, 0.13767257564384233, 0.0016691551050781548, 7.339762376678778e-06, 0.00624595326957781, 0.004479406223211265, 0.004224960270734571, 7.107736771674856e-08, 0.06165363303235661, 2.0952070683687907e-07, 4.47

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:506: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.5924474062459166
precision 0.6587449933244326 Signal D-13
recall 0.9684003925417075 Signal D-13
Accuracy 0.6447144910492617 Signal D-13
F1 0.7841080651569329 Signal D-13
epoch 10 loss:  0.13549070060253143
Training time: 0.5870443344116211 Signal A-9
 Number Of Trainable Parameters 228897 Signal A-9
 Testing Time 0.20117402076721191 Signal A-9
[0.0015538817810600044, 1.504294963669963e-06, 0.016425899428125414, 0.36716248826688885, 0.012948204705798947, 8.87386111541333e-07, 0.00022666060227981947, 0.001966362049421288, 0.018527862588044577, 0.002562994389914008, 0.008975656751227901, 0.0004982309522393031, 0.0026710833156142343, 0.00011056170489142075, 0.000802129676171351, 0.000540648551107249, 2.879360355240949e-06, 0.13767257564384233, 0.0016691551050781548, 7.339762376678778e-06, 0.00624595326957781, 0.004479406223211265, 0.004224960270734571, 7.107736771674856e-08, 0.06165363303235661, 2.0952070683687907e-07, 4.4782

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:506: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.4567901234567901
precision 0.9918806232170287 Signal A-9
recall 0.5390578413834228 Signal A-9
Accuracy 0.5365622032288699 Signal A-9
F1 0.698501004481533 Signal A-9
precision_final 0.8447604585324991
recall_final 0.872782849271751
Accuracy_final 0.7684023358324705
F1_final 0.8391446178673354
MSE_final 0.07750712773666504


In [ ]:
Calculations['pipline']='Bayesian_lstm'
MSE_final=mean(MSE)
print('MSE_final', MSE_final)
import os   
Calculations.to_csv('Calculations.csv')  


MSE_final 0.044757840667867706


In [6]:
import io
import os
import random
import urllib.request
import zipfile
import datetime
import pdb
import holidays
import json
import copy
import time
import numpy as np
import torch
import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from torch.profiler import profile, record_function, ProfilerActivity
from sklearn.model_selection import train_test_split

train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')
seed=42
torch.backends.cudnn.determinestic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


def build_df(data, start=0):
#
    index = np.array(range(start, start + len(data)))
    timestamp = index * 86400 + 1022819200



    return pd.DataFrame({'timestamp': timestamp.astype(int), 'value': data[:, 0], 'index': index.astype(int)})

Calculations = pd.DataFrame()
def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)


def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))
class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):
        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size  # user-defined

        self.hidden_size_1 = 128
        self.hidden_size_2 = 32
        self.stacked_layers = 2
        self.dropout_probability = 0.2  # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True, dropout=self.dropout_probability)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True,dropout=self.dropout_probability)

        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)

        output, _ = self.lstm1(x, hidden)

        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :]  # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']





SMAP=[  'P-1', 'S-1', 'E-1', 'E-2','E-5', 'E-6', 'E-7',
    'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13',   'P-3',
     'A-2', 'A-4', 'G-2',
    'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9',
    'G-4', 'T-3',  'D-12', 'B-1',  'G-7', 'P-7',
     'A-5', 'A-7', 'D-13',  'A-9']

MSL=[
    'M-1', 'S-2',  'T-4', 'T-5', 'F-7',  'M-4',
    'M-5',  'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9',  'T-8',  'D-15', 'M-7', 'F-8']
SIGNALS = [
    'P-1', 'S-1', 'E-1', 'E-2', 'E-3', 'E-4', 'E-5', 'E-6', 'E-7',
    'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13', 'A-1', 'D-1', 'P-3',
    'D-2', 'D-3', 'D-4', 'A-2', 'A-3', 'A-4', 'G-1', 'G-2', 'D-5',
    'D-6', 'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9',
    'F-2', 'G-4', 'T-3', 'D-11',  'B-1', 'G-6', 'G-7', 'P-7',
    'R-1', 'A-5', 'A-6', 'A-7', 'D-13', 'A-8', 'A-9', 'F-3', 'M-6',
    'M-1', 'M-2', 'S-2', 'P-10', 'T-4', 'T-5', 'F-7', 'M-3', 'M-4',
    'M-5', 'P-15', 'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9', 'P-14', 'T-8', 'P-11', 'D-15', 'D-16', 'M-7', 'F-8'
]
SMAP=[  'P-1',  'E-1', 'E-2', 'E-3', 'E-4', 'E-5', 'E-6', 'E-7',
    'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'A-1', 'D-1', 'P-3',
    'D-2', 'D-3', 'D-4', 'A-2', 'A-3', 'A-4', 'G-1', 'G-2', 'D-5',
    'D-6', 'D-7', 'P-4', 'G-3',  'D-8', 'D-9',
    'F-2', 'G-4', 'T-3', 'D-11',  'G-6', 'G-7',
    'R-1', 'A-5', 'A-6',  'D-13', 'A-8', 'F-3']


MSL=['M-6',
    'M-1', 'M-2', 'S-2', 'P-10', 'T-4', 'T-5', 'F-7', 'M-3', 'M-4',
    'M-5', 'P-15', 'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9', 'P-14', 'T-8', 'P-11', 'D-15', 'D-16', 'M-7', 'F-8']
#SMAP=pd.Series(SMAP)
#MSL=pd.Series(MSL)


avg=[]
train_signals=MSL
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()
MSE=[]
df_label=df_label[df_label['chan_id'].isin(MSL)]

df_label=df_label.reset_index(drop=True)
TrainingTime=[]
TestingTime=[]


NumberOfParameters=[]
for name in MSL:
        label_row=df_label[df_label.chan_id==name]
        #pdb.set_trace()
     
        labels= label_row.anomaly_sequences[im]
        labels=eval(labels)
      
        true_indices_grouped = [list(range(e[0], e[1]+1)) for e in labels]
        true_indices_flat = set([i for group in true_indices_grouped for i in group])


        appended_data = []

    
        labels= true_indices_flat
       

       # for i in range(len(labels)):
        #    anom=labels[i]
            #start=anom[0]
            #end=anom[1]

        index = list(labels)

       # timestamp = index * 86400 + 1022819200


        anomalies=pd.DataFrame({'value': 1, 'index':index})
       #     appended_data.append(anomalies)

        label_data = anomalies

        #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
        #label_data ['month'] = label_data ['date'].dt.month.astype(int)
        label_data ['name'] = name
        #label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
        #label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
        label_data  = label_data [selected_columns]
        #label_data .to_csv('csv/' + name + '.csv', index=False)


        signal=name
        train_np = np.load('data/train/' + signal + '.npy')
        test_np = np.load('data/test/' + signal + '.npy')


        data = build_df(np.concatenate([train_np, test_np]))
        data['date'] = pd.to_datetime(data['timestamp'], unit='s')
        data['month'] = data['date'].dt.month.astype(int)
        data['name'] = name
        data['index'] = data['index'].astype(int)
        data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
        data['hour_of_day'] = data['date'].dt.hour.astype(int)
        data = data[selected_columns]
        _ , validation = train_test_split(
            data,
            test_size=0.15
        )

        val_dataset=validation

        val=val_dataset


        #val['index'] = val['index'].astype(int)
        #val['date'] = pd.to_datetime(val['timestamp'], unit='s')
        #val['month'] = val['date'].dt.month.astype(int)
        #val['day_of_month'] = val['date'].dt.day.astype(int)
        #val['name'] = name
        #val['day_of_week'] = val['date'].dt.dayofweek.astype(int)
        #val['hour_of_day'] = val['date'].dt.hour.astype(int)
        #val['index'] = val['index'].astype(int)
        #val = val[selected_columns]

        train = build_df(train_np)
        train['date'] = pd.to_datetime(train['timestamp'], unit='s')
        train['month'] = train['date'].dt.month.astype(int)
        train['day_of_month'] = train['date'].dt.day.astype(int)
        train['name'] = name
        train['day_of_week'] = train['date'].dt.dayofweek.astype(int)
        train['hour_of_day'] = train['date'].dt.hour.astype(int)
        train['index'] = train['index'].astype(int)
        train = train[selected_columns]
        # train.to_csv('csv/' + name + '.csv', index=False)
        # train.to_csv('csv/' + name + '-train.csv', index=False)


        test = build_df(test_np, start=len(train))
        test['date'] = pd.to_datetime(test['timestamp'], unit='s')
        test['month'] = test['date'].dt.month.astype(int)
        test['name'] = name
        test['day_of_week'] = test['date'].dt.dayofweek.astype(int)
        test['hour_of_day'] = test['date'].dt.hour.astype(int)
        test['index'] = test['index'].astype(int)
        test = test[selected_columns]
        # test.to_csv('csv/' + name + '.csv', index=False)
        # test.to_csv('csv/' + name + '-train.csv', index=False)
        # test.to_csv('csv/' + name + '-test.csv', index=False)

        datetime_columns = ['index',  'name']
        target_column = 'value'

        feature_columns = datetime_columns + ['value']

        resample_df = train[feature_columns]
        resample_df_test = test[feature_columns]
        resample_df_val=val[feature_columns]
        #print(resample_df)

       # plot_length = 1000000
        #plot_df = resample_df.copy(deep=True).iloc[:plot_length]
        #plot_df['weekday'] = plot_df['date'].dt.day_name()

        n_train = len(train_np)
        n_test = len(test_np)

        features = ['index',  'value']
        feature_array = resample_df[features].values
        feature_array_test = resample_df_test[features].values
        feature_array_val = resample_df_val[features].values
        # Fit Scaler only on Training features
        feature_scaler = MinMaxScaler()
        feature_scaler.fit(feature_array[:n_train])
        # Fit Scaler only on Training target values
        #feature_scaler.fit(feature_array_test[:n_test])

        target_scaler = MinMaxScaler()
        target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

        # Transfom on both Training and Test data
        scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                                    columns=features)

        scaled_array_test = pd.DataFrame(feature_scaler.transform(feature_array_test),
                                         columns=features)
        scaled_array_val = pd.DataFrame(feature_scaler.transform(feature_array_val),
                                         columns=features)

        sequence_length = 10

        X_train, y_train = create_sliding_window(scaled_array,
                                                 sequence_length)

        X_test, y_test = create_sliding_window(scaled_array_test,
                                               sequence_length)
        X_val, y_val = create_sliding_window(scaled_array_val,
                                               sequence_length)

        #sns.displot(data=y_test,  kind="hist", bins=100, aspect=1.5)
      #  plt.show()
       # pdb.set_trace()

        #f = Fitter(y_train,
         #          distributions=['gamma',
          #                        'lognorm',
           #                       "beta",
            #                      "burr",
             #                     "norm",'cauchy', 'chi2', 'expon', 'exponpow', 'powerlaw', 'rayleigh', 'uniform'])
        #f.fit()
        #f.summary()
        #pdb.set_trace()
        n_features = scaled_array.shape[-1]
        sequence_length = 10
        output_length = 1

        batch_size = 64
        n_epochs = 10
        learning_rate = 0.01

        bayesian_lstm = BayesianLSTM(n_features=n_features,
                                     output_length=output_length,
                                     batch_size=batch_size)
        

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
        bayesian_lstm.train()
        start=time.time()

        for e in range(1, n_epochs + 1):
            for b in range(0, len(X_train), batch_size):
                features = X_train[b:b + batch_size, :, :]
                target = y_train[b:b + batch_size]

                X_batch = torch.tensor(features, dtype=torch.float32)
                y_batch = torch.tensor(target, dtype=torch.float32)
               # pdb.set_trace()

                output = bayesian_lstm(X_batch)
                loss = criterion(output.view(-1), y_batch)


                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if e % 10 == 0:
                print('epoch', e, 'loss: ', loss.item())
                offset = sequence_length

        training_df = pd.DataFrame()
       # training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1]
        training_df['index'] = resample_df['index'].iloc[offset:n_train + offset:1]
        training_predictions = bayesian_lstm.predict(X_train)
        end=time.time()
        NumberOfTrainableParameters=sum(p.numel() for p in bayesian_lstm.parameters() if p.requires_grad)
        TrainingingTime=(end-start)/n_epochs
        NumberOfParameters.append(NumberOfTrainableParameters)
        TrainingTime.append(TrainingingTime)
        MSE.append(loss.item())

 



        print("Training time:",(end-start)/n_epochs,'Signal', name)
    
        print(" Number Of Trainable Parameters", NumberOfTrainableParameters,'Signal', name)
   


        training_df['value'] = inverse_transform(training_predictions)
        training_df['source'] = 'Training Prediction'


        #training_truth_df['date'] = training_df['date']
        training_truth_df['index'] = training_df['index']
        training_truth_df['value'] = resample_df['value'].iloc[
                                     offset:n_train + offset:1]
        training_truth_df['source'] = 'True Values'

        testing_df = pd.DataFrame()
       # testing_df['date'] = resample_df_test['date'].iloc[offset:n_test + offset:1]
        testing_df['index'] = resample_df_test['index'].iloc[offset:n_test + offset:1]
        start=time.time()

        testing_predictions = bayesian_lstm.predict(X_test)
        end=time.time()
        TestingingTime=(end-start)/n_epochs
        print(" Testing Time", (end-start)/n_epochs,'Signal', name)
        TestingTime.append(TestingingTime)

        #print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


        
                     
      




        testing_df['value'] = inverse_transform(testing_predictions)
        testing_df['source'] = 'Test Prediction'

        testing_truth_df = pd.DataFrame()
        #testing_truth_df['date'] = testing_df['date']
        testing_truth_df['index'] = testing_df['index']

        testing_truth_df['value'] = resample_df_test['value'].iloc[offset:n_test + offset:1]
        testing_truth_df['source'] = 'True Values'


        evaluation = pd.concat([training_df,
                                testing_df,
                                training_truth_df,
                                testing_truth_df
                                ], axis=0)

        fig = px.line(evaluation,
                             x="index",
                              y="value",
                              color="source")
        #fig.show()

        n_experiments = 2
        test_uncertainty_df = pd.DataFrame()
        #test_uncertainty_df['date'] = testing_df['date']
        test_uncertainty_df['index'] = testing_df['index']

        for i in range(n_experiments):
            experiment_predictions = bayesian_lstm.predict(X_test)

            test_uncertainty_df['value_{}'.format(i)] = inverse_transform(experiment_predictions)

        log_energy_consumption_df = test_uncertainty_df.filter(like='value', axis=1)
        test_uncertainty_df['value_mean'] = log_energy_consumption_df.mean(axis=1)
        test_uncertainty_df['value_std'] = log_energy_consumption_df.std(axis=1)


        test_uncertainty1 = test_uncertainty_df['value_mean']

        #########################Adaptive Smoothing

        #   pdb.set_trace()
        Threshold = 10
        gamma = 0.1
        betta = 0.1
        test_uncertainty2 = []
        ###########
        for jj in range(len(test_uncertainty1) - 2):
            test_uncertaintyy = (1 / (1 + gamma + betta)) * (
                        test_uncertainty1.iloc[jj] + gamma * test_uncertainty1.iloc[jj + 1] + betta *
                        test_uncertainty1.iloc[jj + 2])
            test_uncertainty2.append(test_uncertaintyy)
        test_uncertainty2 = pd.DataFrame(test_uncertainty2)

        test_uncertainty2.columns = ["value_mean"]


        test_uncertainty_df["value_mean"] = test_uncertainty2["value_mean"]

        ########################################################


        test_uncertainty_df = test_uncertainty_df[['index',  'value_mean', 'value_std']]
        test_uncertainty_df['lower_bound'] = test_uncertainty_df['value_mean'] - 5* test_uncertainty_df['value_std']
        test_uncertainty_df['upper_bound'] = test_uncertainty_df['value_mean'] + 5 * test_uncertainty_df['value_std']
        import plotly.graph_objects as go

        test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
        # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
        truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
        # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

        upper_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['upper_bound'],
          mode='lines',
         fill=None,
         name='99% Upper Confidence Bound'
         )
        lower_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['lower_bound'],
          mode='lines',
          fill='tonexty',
         name= '99% Lower Confidence Bound',
         fillcolor='rgba(255, 211, 0, 0.1)',
         )
        real_trace = go.Scatter(
            x=truth_uncertainty_plot_df['index'],
            y=truth_uncertainty_plot_df['value'],
            mode='lines',
            fill=None,
            name='Real Values'
        )

        labels = go.Scatter(
           x=label_data['index'],
          y=label_data['value'],
         mode='lines',
         fill='tonexty' ,
         name='labels'
         )
        data = [upper_trace, lower_trace, real_trace]

        fig = go.Figure(data=data)
        fig.update_layout(title='Uncertainty MCDropout Test Data',
                            xaxis_title='index',
                            yaxis_title='value',
         legend_font_size=14,
                          )
        #fig.show()
        bounds_df = pd.DataFrame()

        # Using 99% confidence bounds
        bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
        bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
        bounds_df['real_value'] = truth_uncertainty_plot_df['value']
        bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

        bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                                  (bounds_df['real_value'] <= bounds_df['upper_bound']))

        print("Proportion of points contained within 99% confidence interval:",
              bounds_df['contained'].mean())
        predictedanomaly = bounds_df.index[~bounds_df['contained']]

        N = 15
        newarr = []


        for i in range(len(predictedanomaly) - N):
            if (predictedanomaly[i] + 1 == predictedanomaly[i + 1] and predictedanomaly[i + 1] + 1 == predictedanomaly[
               i + 2] and predictedanomaly[i+3]+1==predictedanomaly[i+4] and predictedanomaly[i+4]+1==predictedanomaly[i+5]
                and  predictedanomaly[i+5]+1==predictedanomaly[i+6] and predictedanomaly[i+6]+1==predictedanomaly[i+7]
                and predictedanomaly[i+7]+1==predictedanomaly[i+8]
                and predictedanomaly[i+8]+1==predictedanomaly[i+9]
                and predictedanomaly[i+9]+1==predictedanomaly[i+10]
                and predictedanomaly[i+10]+1==predictedanomaly[i+11]
                and predictedanomaly[i+11]+1==predictedanomaly[i+12]
                and predictedanomaly[i + 12] + 1 == predictedanomaly[i + 13]
                and predictedanomaly[i + 13] + 1 == predictedanomaly[i + 14]
                and predictedanomaly[i + 14] + 1 == predictedanomaly[i + 15]):

                newarr.append(predictedanomaly[i])
        #        newarr.append(predictedanomaly[i + 1])
         #       newarr.append(predictedanomaly[i + 2])


        predicteddanomaly = list(set(newarr))
        MSE.append(mean( testing_truth_df['value']-test_uncertainty_df['value_mean'])**2)
        print(MSE)





        realanomaly = label_data['index']

        predicter=list(range(len(test_uncertainty2)))

        a1 = pd.DataFrame(index=range(len(test_uncertainty2)),columns=range(2))
        a1.columns=['index','value']

        a2 = pd.DataFrame(index=range(len(test_uncertainty2)), columns=range(2))
        a2.columns = ['index','value']




        for i in range(len(predicter)):
            if i in predicteddanomaly:
                a1.iloc[i,1]=1
            else:
                a1.iloc[i,1]=0

        for i in range(len(predicter)):
            if i in realanomaly:
                a2.iloc[i,1] = 1
            else:
                a2.iloc[i, 1] = 0



        y_real=a2.value
        y_real=y_real.astype(int)
        y_predi=a1.value
        y_predi=y_predi.astype(int)


        cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
        cm_plot_labels = ['no_anomaly', 'had_anomaly']
        plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

        # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
        #fp = len(predicteddanomaly) - tp
        #fn = 0
        #tn = len(truth_uncertainty_plot_df) - tp - fp - fn

        tp=cm[0][0]
        fp=cm[0][1]
        fn=cm[1][0]
        tn=cm[1][1]

        precision1 = tp / (tp + fp)
        recall1 = tp / (tp + fn)
        Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
        F11 = 2 / ((1 / precision1) + (1 / recall1))
        print('precision', precision1, 'Signal', name)
        print('recall', recall1, 'Signal', name)
        print('Accuracy', Accuracy1, 'Signal', name)
        print('F1', F11, 'Signal', name)
        precision.append(precision1)
        F1.append(F11)
        Accuracy.append(Accuracy1)
        recall.append(recall1)

        im=im+1



        #matched_indices = list(i_anom_predicted & true_indices_flat)

Calculations['pipline']='Bayesian_LSTM'
Calculations['fit_time']=TrainingTime
Calculations['prediction_time']=TestingTime

#Calculations.iloc['number_of_parameters']=NumberOfTrainableParameters
#recall_final=mean(recall)
#precision_final=mean(precision)
#F1_final=mean(F1)
#Accuracy_final=mean(Accuracy)
#cm = confusion_matrix(y_true=test_labels, y_pred=predicteddanomaly)

################################################################################

# %%
#recall_final=mean(recall)
#precision_final=mean(precision)
#F1_final=mean(F1)
MSE_final=mean(MSE)
#Accuracy_final=mean(Accuracy)
#print('precision_final', precision_final)
#print('recall_final', recall_final)
#print('Accuracy_final',Accuracy_final)
print('F1_final', F1_final)
print('MSE_final', MSE_final)
## New Test Bayesian LSTM AE



epoch 10 loss:  4.817243734578369e-06
Training time: 1.9776106357574463 Signal M-6
 Number Of Trainable Parameters 228897 Signal M-6
 Testing Time 0.06991562843322754 Signal M-6


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:529: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.4561059342815105
[4.817243734578369e-06, nan]
precision 0.4482758620689655 Signal M-6
recall 0.8213228035538006 Signal M-6
Accuracy 0.40804315841098576 Signal M-6
F1 0.5799930289299408 Signal M-6
epoch 10 loss:  0.0333426371216774
Training time: 1.7403138875961304 Signal M-1
 Number Of Trainable Parameters 228897 Signal M-1
 Testing Time 0.070931077003479 Signal M-1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:529: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.026025584472871635
[4.817243734578369e-06, nan, 0.0333426371216774, nan]
precision 0.19217081850533807 Signal M-1
recall 0.9557522123893806 Signal M-1
Accuracy 0.5941773268636965 Signal M-1
F1 0.32 Signal M-1
epoch 10 loss:  0.14085425436496735
Training time: 1.9433006525039673 Signal M-2
 Number Of Trainable Parameters 228897 Signal M-2
 Testing Time 0.07765352725982666 Signal M-2


...

Proportion of points contained within 99% confidence interval: 0.10408069692801467
[4.817243734578369e-06, nan, 0.0333426371216774, nan, 0.14085425436496735, nan, 6.565194780705497e-05, nan, 0.005667318124324083, nan, 0.2324029803276062, nan, 2.5438855573156616e-06, nan, 0.010235900059342384, nan, 0.04033857211470604, nan, 0.06876510381698608, nan, 0.06498318910598755, nan, 0.0038100939709693193, nan, 0.005871171597391367, nan, 4.5980195864103734e-05, nan, 0.16260473430156708, nan, 0.14057578146457672, nan, 0.03005399741232395, nan, 1.4874936823616736e-05, nan, 7.834634629944048e-07, nan, 0.004319627303630114, nan, 0.008534700609743595, nan, 0.0001280488504562527, nan, 0.007573776878416538, nan, 0.17942987382411957, nan, 0.03682702034711838, nan]
precision 0.281413612565445 Signal D-16
recall 0.8037383177570093 Signal D-16
Accuracy 0.44750114626318205 Signal D-16
F1 0.4168686379059621 Signal D-16
epoch 10 loss:  0.00015790079487487674
Training time: 1.5452480554580688 Signal M-7
 Numbe

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:529: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.5773532152842498
[4.817243734578369e-06, nan, 0.0333426371216774, nan, 0.14085425436496735, nan, 6.565194780705497e-05, nan, 0.005667318124324083, nan, 0.2324029803276062, nan, 2.5438855573156616e-06, nan, 0.010235900059342384, nan, 0.04033857211470604, nan, 0.06876510381698608, nan, 0.06498318910598755, nan, 0.0038100939709693193, nan, 0.005871171597391367, nan, 4.5980195864103734e-05, nan, 0.16260473430156708, nan, 0.14057578146457672, nan, 0.03005399741232395, nan, 1.4874936823616736e-05, nan, 7.834634629944048e-07, nan, 0.004319627303630114, nan, 0.008534700609743595, nan, 0.0001280488504562527, nan, 0.007573776878416538, nan, 0.17942987382411957, nan, 0.03682702034711838, nan, 0.00015790079487487674, nan]
precision 0.9813999021047479 Signal M-7
recall 0.9520417853751187 Signal M-7
Accuracy 0.9342963653308481 Signal M-7
F1 0.9664979513135695 Signal M-7
epoch 10 loss:  0.08980521559715271
Training time: 3.4363221645355

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:529: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.7242632216390795
[4.817243734578369e-06, nan, 0.0333426371216774, nan, 0.14085425436496735, nan, 6.565194780705497e-05, nan, 0.005667318124324083, nan, 0.2324029803276062, nan, 2.5438855573156616e-06, nan, 0.010235900059342384, nan, 0.04033857211470604, nan, 0.06876510381698608, nan, 0.06498318910598755, nan, 0.0038100939709693193, nan, 0.005871171597391367, nan, 4.5980195864103734e-05, nan, 0.16260473430156708, nan, 0.14057578146457672, nan, 0.03005399741232395, nan, 1.4874936823616736e-05, nan, 7.834634629944048e-07, nan, 0.004319627303630114, nan, 0.008534700609743595, nan, 0.0001280488504562527, nan, 0.007573776878416538, nan, 0.17942987382411957, nan, 0.03682702034711838, nan, 0.00015790079487487674, nan, 0.08980521559715271, nan]
precision 1.0 Signal F-8
recall 0.7830303030303031 Signal F-8
Accuracy 0.7823980621719823 Signal F-8
F1 0.8783140720598233 Signal F-8
F1_final 0.8391446178673354
MSE_final nan


In [23]:
MSE1=MSE.copy() 
import math
from math import sqrt
newlist = [x for x in MSE1 if math.isnan(x) == False]
MSE_final=sqrt(mean(newlist))
MSE_final


0.21657404427612753

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Calculations['pipline']='Bayesian_lstm'
MSE_final=mean(MSE)
print('MSE_final', MSE_final)
import os   
Calculations.to_csv('Calculations2.csv')  

MSE_final 0.046904316654118046


In [ ]:
import pdb


import pandas as pd

import logging
import warnings
import statistics
pipeline = 'arima'
logging.basicConfig(level=logging.ERROR)
logging.getLogger().setLevel(level=logging.ERROR)
warnings.simplefilter("ignore")
SIGNALS = [
    'P-1', 'S-1', 'E-1', 'E-2', 'E-3', 'E-4', 'E-5', 'E-6', 'E-7',
    'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13', 'A-1', 'D-1', 'P-3',
    'D-2', 'D-3', 'D-4', 'A-2', 'A-3', 'A-4', 'G-1', 'G-2', 'D-5',
    'D-6', 'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9',
    'F-2', 'G-4', 'T-3', 'D-11', 'D-12', 'B-1', 'G-6', 'G-7', 'P-7',
    'R-1', 'A-5', 'A-6', 'A-7', 'D-13', 'A-8', 'A-9', 'F-3', 'M-6',
    'M-1', 'M-2', 'S-2', 'P-10', 'T-4', 'T-5', 'F-7', 'M-3', 'M-4',
    'M-5', 'P-15', 'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9', 'P-14', 'T-8', 'P-11', 'D-15', 'D-16', 'M-7', 'F-8'
]
SMAP=[  'P-1', 'S-1', 'E-1', 'E-2', 'E-3', 'E-4', 'E-5', 'E-6', 'E-7',
    'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13', 'A-1', 'D-1', 'P-3',
    'D-2', 'D-3', 'D-4', 'A-2', 'A-3', 'A-4', 'G-1', 'G-2', 'D-5',
    'D-6', 'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9',
    'F-2', 'G-4', 'T-3', 'D-11', 'D-12', 'B-1', 'G-6', 'G-7', 'P-7',
    'R-1', 'A-5', 'A-6', 'A-7', 'D-13', 'A-8', 'A-9', 'F-3']



MSL=['M-6',
    'M-1', 'M-2', 'S-2', 'P-10', 'T-4', 'T-5', 'F-7', 'M-3', 'M-4',
    'M-5', 'P-15', 'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9', 'P-14', 'T-8', 'P-11', 'D-15', 'D-16', 'M-7', 'F-8']

append_train='-train'
append_test='-test'
append_new='-new'
training = [sub+append_train for sub in SMAP]



from statsmodels.tsa.arima_model import ARIMA
import io
import os
import random
import urllib.request
import zipfile
import datetime
import pdb

import json
import copy
import time
import numpy as np

import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions

from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools

import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from torch.profiler import profile, record_function, ProfilerActivity
from sklearn.model_selection import train_test_split
import time
train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')

###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']







#SMAP=pd.Series(SMAP)
#MSL=pd.Series(MSL)
from math import sqrt
from sklearn.metrics import mean_squared_error

avg=[]
train_signals=SMAP
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()
MSE=[]
df_label=df_label[df_label['chan_id'].isin(SMAP)]

df_label=df_label.reset_index(drop=True)
TrainingTime=[]
TestingTime=[]
signal='S-1'
NumberOfParameters=[]
RMSE=[]
#import autopep8

Train_t=[]
Predict_t=[]
counter=0
for signal in SMAP:
  train_np = np.load('data/train/' + signal + '.npy')
  test_np = np.load('data/test/' + signal + '.npy')
  train =train_np[:,0]
  test  = test_np[:,0]
  # order = (p=1, d=1, q=1)
  history = [x for x in train]
  predictions = list()
  # walk-forward validation
  T_fit=[]
  T_pred=[]

  for t in range(25):
    try:
     model = ARIMA(history, order=(2,1,2))

     start = time.time()
     model_fit = model.fit()
     end=time.time()
     T_fit.append(end-start)

    
     start=time.time()
     output = model_fit.forecast()
     end=time.time()
     T_pred.append(end-start)
     yhat = output[0].item()
     predictions.append(yhat)
     obs = test[t]
    
    
     history.append(obs)
    except:
      counter=counter+1
      pass

  try:    
   T11=mean(T_fit)
   T12=mean(T_pred)
 
   print("fit_time",T11,'signal',signal)
   print("prediction_time",T12,'signal',signal)

  #print('predicted=%f, expected=%f' % (yhat, obs))
  #print(model_fit.maparams)
  #pdb.set_trace()
   rmse = sqrt(mean_squared_error(test[0:25], predictions))
   print('Test RMSE: %.3f' % rmse,'signal',signal)
   RMSE.append(sqrt(rmse))
   
   Train_t.append( T11)
   Predict_t.append(T12)
  except:
   rmse = 1
   print('Error signal',signal)
   RMSE.append(sqrt(rmse))
  # plot forecasts against actual outcomes
  #pyplot.plot(test)
#pyplot.plot(predictions, color='red')
#pyplot.show()
#end=time.time()
# Make as pandas series
#result = model.get_forecast()
df_SMAP=pd.DataFrame()
df_SMAP['fit_time']=Train_t
df_SMAP['Prediction_time']=Predict_t
df_SMAP['pipline']='Arima'
from math import sqrt
MSE_final=mean(RMSE)
print('MSE_final', MSE_final)
import os   
df_SMAP.to_csv('ARIMA_SMAP.csv')  



fit_time 1.7132806587219238 signal P-1
prediction_time 0.003330793380737305 signal P-1
Test RMSE: 0.276 signal P-1


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


fit_time 3.3255919933319094 signal S-1
prediction_time 0.0043089199066162105 signal S-1
Test RMSE: 0.379 signal S-1
fit_time 1.2583719062805176 signal E-1
prediction_time 0.00346003532409668 signal E-1
Test RMSE: 0.055 signal E-1
fit_time 0.5033388710021973 signal E-2
prediction_time 0.001546649932861328 signal E-2
Test RMSE: 0.022 signal E-2
fit_time 0.9497098541259765 signal E-3
prediction_time 0.003185291290283203 signal E-3
Test RMSE: 0.154 signal E-3
fit_time 2.1476333332061768 signal E-4
prediction_time 0.00412419319152832 signal E-4
Test RMSE: 0.066 signal E-4
fit_time 1.243541316986084 signal E-5
prediction_time 0.0032325077056884768 signal E-5
Test RMSE: 0.060 signal E-5


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist

fit_time 1.9264053630828857 signal E-6
prediction_time 0.004890375137329102 signal E-6
Test RMSE: 0.000 signal E-6
fit_time 0.6229221534729004 signal E-7
prediction_time 0.0015096664428710938 signal E-7
Test RMSE: 0.040 signal E-7
fit_time 0.7938701629638671 signal E-8
prediction_time 0.0021731376647949217 signal E-8
Test RMSE: 0.118 signal E-8
fit_time 1.1007671737670899 signal E-9
prediction_time 0.0034842300415039063 signal E-9
Test RMSE: 0.128 signal E-9
fit_time 1.5434263134002686 signal E-10
prediction_time 0.003950128555297852 signal E-10
Test RMSE: 0.037 signal E-10


...

fit_time 1.0722429370880127 signal G-7
prediction_time 0.0035628318786621095 signal G-7
Test RMSE: 0.013 signal G-7
fit_time 0.8981855487823487 signal P-7
prediction_time 0.0030879974365234375 signal P-7
Test RMSE: 0.067 signal P-7
Error signal R-1


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist

fit_time 0.39312106132507324 signal A-5
prediction_time 0.0017525959014892578 signal A-5
Test RMSE: 0.000 signal A-5
fit_time 0.8048231506347656 signal A-6
prediction_time 0.0017430496215820313 signal A-6
Test RMSE: 0.198 signal A-6
Error signal A-7
Error signal D-13
fit_time 1.2838919258117676 signal P-2
prediction_time 0.001996622085571289 signal P-2
Test RMSE: 0.026 signal P-2
Error signal A-8
Error signal A-9


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


fit_time 0.5120196342468262 signal F-3
prediction_time 0.0024671554565429688 signal F-3
Error signal F-3
MSE_final 0.5426334643017061


In [ ]:
print(mean(df_SMAP.fit_time))
print(mean(df_SMAP.Prediction_time))

1.2868510804456823
0.003014413609224207


In [ ]:
len(SMAP)

55

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import io
import os
import random
import urllib.request
import zipfile
import datetime
import pdb

import json
import copy
import time
import numpy as np

import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions

from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools

import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from torch.profiler import profile, record_function, ProfilerActivity
from sklearn.model_selection import train_test_split
import time
train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')

###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']







#SMAP=pd.Series(SMAP)
#MSL=pd.Series(MSL)
from math import sqrt
from sklearn.metrics import mean_squared_error

avg=[]
train_signals=MSL
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()
MSE=[]
df_label=df_label[df_label['chan_id'].isin(MSL)]

df_label=df_label.reset_index(drop=True)
TrainingTime=[]
TestingTime=[]
signal='S-1'
NumberOfParameters=[]
RMSE=[]
#import autopep8

Train_t=[]
Predict_t=[]
counter=0
for signal in MSL:
  train_np = np.load('data/train/' + signal + '.npy')
  test_np = np.load('data/test/' + signal + '.npy')
  train =train_np[:,0]
  test  = test_np[:,0]
  # order = (p=1, d=1, q=1)
  history = [x for x in train]
  predictions = list()
  # walk-forward validation
  T_fit=[]
  T_pred=[]

  for t in range(25):
    try:
     model = ARIMA(history, order=(3,1,8))

     start = time.time()
     model_fit = model.fit()
     end=time.time()
     T_fit.append(end-start)

    
     start=time.time()
     output = model_fit.forecast()
     end=time.time()
     T_pred.append(end-start)
     yhat = output[0].item()
     predictions.append(yhat)
     obs = test[t]
    
    
     history.append(obs)
    except:
      counter=counter+1
      pass

  try:    
   T11=mean(T_fit)
   T12=mean(T_pred)
 
   print("fit_time",T11,'signal',signal)
   print("prediction_time",T12,'signal',signal)

  #print('predicted=%f, expected=%f' % (yhat, obs))
  #print(model_fit.maparams)
  #pdb.set_trace()
   rmse = sqrt(mean_squared_error(test[0:25], predictions))
   print('Test RMSE: %.3f' % rmse,'signal',signal)
   RMSE.append(sqrt(rmse))
   
   Train_t.append( T11)
   Predict_t.append(T12)
  except:
   rmse = 1
   print('Error signal',signal)
   RMSE.append(sqrt(rmse))
  # plot forecasts against actual outcomes
  #pyplot.plot(test)
#pyplot.plot(predictions, color='red')
#pyplot.show()
#end=time.time()
# Make as pandas series
#result = model.get_forecast()
df_MSL=pd.DataFrame()
df_MSL['fit_time']=Train_t
df_MSL['Prediction_time']=Predict_t
df_MSL['pipline']='Arima'
from math import sqrt
MSE_final=mean(RMSE)
print('MSE_final', MSE_final)
import os   
df_MSL.to_csv('ARIMA_MSL.csv')  

Error signal M-6
Error signal M-1
Error signal M-2
Error signal S-2


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist

fit_time 17.743872079849243 signal P-10
prediction_time 0.021655302047729492 signal P-10
Test RMSE: 0.001 signal P-10
fit_time 15.793594274520874 signal T-4
prediction_time 0.00856522560119629 signal T-4
Test RMSE: 0.390 signal T-4
Error signal T-5


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/pyt

fit_time 77.06782509803772 signal F-7
prediction_time 0.014726800918579102 signal F-7
Test RMSE: 0.220 signal F-7
Error signal M-3
fit_time 8.584167985916137 signal M-4
prediction_time 0.006396150588989258 signal M-4
Test RMSE: 0.288 signal M-4
fit_time 42.80859560012817 signal M-5
prediction_time 0.00919708251953125 signal M-5
Test RMSE: 0.163 signal M-5
Error signal P-15
Error signal C-1
Error signal C-2
Error signal T-12
Error signal T-13
Error signal F-4
Error signal F-5
Error signal D-14
Error signal T-9


...

fit_time 12.04100227355957 signal P-14
prediction_time 0.015610218048095703 signal P-14
Error signal P-14


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist

fit_time 20.604829111099242 signal T-8
prediction_time 0.007079067230224609 signal T-8
Test RMSE: 0.305 signal T-8
Error signal P-11
fit_time 24.61271692276001 signal D-15
prediction_time 0.007011814117431641 signal D-15
Test RMSE: 0.686 signal D-15
Error signal D-16


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist

fit_time 56.9299391746521 signal M-7
prediction_time 0.010828866958618164 signal M-7
Test RMSE: 0.004 signal M-7
Error signal F-8
MSE_final 0.8334442615922236


In [ ]:
print(mean(df_MSL.fit_time))
print(mean(df_MSL.Prediction_time))

1.8294272035360337
0.004618970155715943


In [ ]:
print(model_fit.summary2())

                          Results: ARIMA
Model:              ARIMA            BIC:                 3752.9220
Dependent Variable: D2.y             Log-Likelihood:      -1844.7  
Date:               2022-05-30 21:42 Scale:               1.0000   
No. Observations:   2817             Method:              css-mle  
Df Model:           7                Sample:              2        
Df Residuals:       2810                                  9        
Converged:          1.0000           S.D. of innovations: 0.465    
No. Iterations:     46.0000          HQIC:                3722.532 
AIC:                3705.3745                                      
--------------------------------------------------------------------
              Coef.   Std.Err.      t       P>|t|    [0.025   0.975]
--------------------------------------------------------------------
const         0.0000    0.0000      0.0625  0.9502  -0.0000   0.0000
ar.L1.D2.y   -0.9446    0.0188    -50.2449  0.0000  -0.9815  -0.9078
ar

In [ ]:
for name in MSL:
        label_row=df_label[df_label.chan_id==name]
        #pdb.set_trace()
     
        labels= label_row.anomaly_sequences[im]
        labels=eval(labels)
      
        true_indices_grouped = [list(range(e[0], e[1]+1)) for e in labels]
        true_indices_flat = set([i for group in true_indices_grouped for i in group])


        appended_data = []

    
        labels= true_indices_flat
       

       # for i in range(len(labels)):
        #    anom=labels[i]
            #start=anom[0]
            #end=anom[1]

        index = list(labels)

       # timestamp = index * 86400 + 1022819200


        anomalies=pd.DataFrame({'value': 1, 'index':index})
       #     appended_data.append(anomalies)

        label_data = anomalies

        #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
        #label_data ['month'] = label_data ['date'].dt.month.astype(int)
        label_data ['name'] = name
        #label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
        #label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
        label_data  = label_data [selected_columns]
        #label_data .to_csv('csv/' + name + '.csv', index=False)


        signal=name
        train_np = np.load('data/train/' + signal + '.npy')
        test_np = np.load('data/test/' + signal + '.npy')


        data = build_df(np.concatenate([train_np, test_np]))
        data['date'] = pd.to_datetime(data['timestamp'], unit='s')
        data['month'] = data['date'].dt.month.astype(int)
        data['name'] = name
        data['index'] = data['index'].astype(int)
        data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
        data['hour_of_day'] = data['date'].dt.hour.astype(int)
        data = data[selected_columns]
        _ , validation = train_test_split(
            data,
            test_size=0.15
        )

        val_dataset=validation

        val=val_dataset


        #val['index'] = val['index'].astype(int)
        #val['date'] = pd.to_datetime(val['timestamp'], unit='s')
        #val['month'] = val['date'].dt.month.astype(int)
        #val['day_of_month'] = val['date'].dt.day.astype(int)
        #val['name'] = name
        #val['day_of_week'] = val['date'].dt.dayofweek.astype(int)
        #val['hour_of_day'] = val['date'].dt.hour.astype(int)
        #val['index'] = val['index'].astype(int)
        #val = val[selected_columns]

        train = build_df(train_np)
        train['date'] = pd.to_datetime(train['timestamp'], unit='s')
        train['month'] = train['date'].dt.month.astype(int)
        train['day_of_month'] = train['date'].dt.day.astype(int)
        train['name'] = name
        train['day_of_week'] = train['date'].dt.dayofweek.astype(int)
        train['hour_of_day'] = train['date'].dt.hour.astype(int)
        train['index'] = train['index'].astype(int)
        train = train[selected_columns]
        # train.to_csv('csv/' + name + '.csv', index=False)
        # train.to_csv('csv/' + name + '-train.csv', index=False)


        test = build_df(test_np, start=len(train))
        test['date'] = pd.to_datetime(test['timestamp'], unit='s')
        test['month'] = test['date'].dt.month.astype(int)
        test['name'] = name
        test['day_of_week'] = test['date'].dt.dayofweek.astype(int)
        test['hour_of_day'] = test['date'].dt.hour.astype(int)
        test['index'] = test['index'].astype(int)
        test = test[selected_columns]
        # test.to_csv('csv/' + name + '.csv', index=False)
        # test.to_csv('csv/' + name + '-train.csv', index=False)
        # test.to_csv('csv/' + name + '-test.csv', index=False)

        datetime_columns = ['index',  'name']
        target_column = 'value'

        feature_columns = datetime_columns + ['value']

        resample_df = train[feature_columns]
        resample_df_test = test[feature_columns]
        resample_df_val=val[feature_columns]
        #print(resample_df)

       # plot_length = 1000000
        #plot_df = resample_df.copy(deep=True).iloc[:plot_length]
        #plot_df['weekday'] = plot_df['date'].dt.day_name()

        n_train = len(train_np)
        n_test = len(test_np)

        features = ['index',  'value']
        feature_array = resample_df[features].values
        feature_array_test = resample_df_test[features].values
        feature_array_val = resample_df_val[features].values
        # Fit Scaler only on Training features
        feature_scaler = MinMaxScaler()
        feature_scaler.fit(feature_array[:n_train])
        # Fit Scaler only on Training target values
        #feature_scaler.fit(feature_array_test[:n_test])

        target_scaler = MinMaxScaler()
        target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

        # Transfom on both Training and Test data
        scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                                    columns=features)

        scaled_array_test = pd.DataFrame(feature_scaler.transform(feature_array_test),
                                         columns=features)
        scaled_array_val = pd.DataFrame(feature_scaler.transform(feature_array_val),
                                         columns=features)

        sequence_length = 10

        X_train, y_train = create_sliding_window(scaled_array,
                                                 sequence_length)

        X_test, y_test = create_sliding_window(scaled_array_test,
                                               sequence_length)
        X_val, y_val = create_sliding_window(scaled_array_val,
                                               sequence_length)

##Results For MSL Signals





MCD-SMAP

In [ ]:
import io
import os
import urllib.request
import zipfile
import datetime
import pdb
import holidays
import json
import copy
import numpy as np
import torch
import random
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools
seed=42
torch.backends.cudnn.determinestic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
import time
train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
       
  
def build_df(data, start=0):
#
    index = np.array(range(start, start + len(data)))
    timestamp = index * 86400 + 1022819200



    return pd.DataFrame({'timestamp': timestamp.astype(int), 'value': data[:, 0], 'index': index.astype(int)})


def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)


def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))
class MCD_AE(nn.Module):

    def __init__(self, n_features, output_length, batch_size):
        super(MCD_AE, self).__init__()

        self.batch_size = batch_size  # user-defined

        self.hidden_size_1 = 128
        self.hidden_size_2 = 64
        self.hidden_size_3= 16
        self.hidden_size_4 = 64
        self.stacked_layers = 3
        self.dropout_probability = 0.5  # arbitrary value (the paper suggests that performance is generally stable across all ranges)


        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm3 = nn.LSTM(self.hidden_size_2,
                             self.hidden_size_3,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm4 = nn.LSTM(self.hidden_size_3,
                             self.hidden_size_4,
                             num_layers=self.stacked_layers,
                             batch_first=True)

        self.fc = nn.Linear(self.hidden_size_4, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)

        output, _ = self.lstm1(x, hidden)

        #output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        state = self.init_hidden3(batch_size)
        output, state = self.lstm3(output, state)
        state = self.init_hidden4(batch_size)
        output, state = self.lstm4(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :]  # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state
    def init_hidden3(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_3))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_3))
        return hidden_state, cell_state
    def init_hidden4(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_4))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_4))
        return hidden_state, cell_state
    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()
###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']






SMAP=pd.Series(SMAP)
MSL=pd.Series(MSL)

#pdb.set_trace()
avg=[]
train_signals=SMAP
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()
T=[]
T2=[]
for name in SMAP:

        label_row = df_label[df_label.chan_id == name]
    # pdb.set_trace()

        labels = label_row.anomaly_sequences[im]
        labels = eval(labels)

        true_indices_grouped = [list(range(e[0], e[1] + 1)) for e in labels]
        true_indices_flat = set([i for group in true_indices_grouped for i in group])

        appended_data = []

        labels = true_indices_flat

    # for i in range(len(labels)):
    #    anom=labels[i]
    # start=anom[0]
    # end=anom[1]

        index = list(labels)

    # timestamp = index * 86400 + 1022819200

        anomalies = pd.DataFrame({'value': 1, 'index': index})
    #     appended_data.append(anomalies)

        label_data = anomalies

    # label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
    # label_data ['month'] = label_data ['date'].dt.month.astype(int)
        label_data['name'] = name
    # label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
    # label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)


        #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
        #label_data ['month'] = label_data ['date'].dt.month.astype(int)
        label_data ['name'] = name
        #label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
        #label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
        label_data  = label_data [selected_columns]
        #label_data .to_csv('csv/' + name + '.csv', index=False)


        signal=name
        train_np = np.load('data/train/' + signal + '.npy')
        test_np = np.load('data/test/' + signal + '.npy')


        data = build_df(np.concatenate([train_np, test_np]))
        data['date'] = pd.to_datetime(data['timestamp'], unit='s')
        data['month'] = data['date'].dt.month.astype(int)
        data['name'] = name
        data['index'] = data['index'].astype(int)
        data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
        data['hour_of_day'] = data['date'].dt.hour.astype(int)
        data = data[selected_columns]
        _ , validation = train_test_split(
            data,
            test_size=0.15
        )

        val_dataset=validation

        val=val_dataset


        #val['index'] = val['index'].astype(int)
        #val['date'] = pd.to_datetime(val['timestamp'], unit='s')
        #val['month'] = val['date'].dt.month.astype(int)
        #val['day_of_month'] = val['date'].dt.day.astype(int)
        #val['name'] = name
        #val['day_of_week'] = val['date'].dt.dayofweek.astype(int)
        #val['hour_of_day'] = val['date'].dt.hour.astype(int)
        #val['index'] = val['index'].astype(int)
        #val = val[selected_columns]

        train = build_df(train_np)
        train['date'] = pd.to_datetime(train['timestamp'], unit='s')
        train['month'] = train['date'].dt.month.astype(int)
        train['day_of_month'] = train['date'].dt.day.astype(int)
        train['name'] = name
        train['day_of_week'] = train['date'].dt.dayofweek.astype(int)
        train['hour_of_day'] = train['date'].dt.hour.astype(int)
        train['index'] = train['index'].astype(int)
        train = train[selected_columns]
        # train.to_csv('csv/' + name + '.csv', index=False)
        # train.to_csv('csv/' + name + '-train.csv', index=False)


        test = build_df(test_np, start=len(train))
        test['date'] = pd.to_datetime(test['timestamp'], unit='s')
        test['month'] = test['date'].dt.month.astype(int)
        test['name'] = name
        test['day_of_week'] = test['date'].dt.dayofweek.astype(int)
        test['hour_of_day'] = test['date'].dt.hour.astype(int)
        test['index'] = test['index'].astype(int)
        test = test[selected_columns]
        # test.to_csv('csv/' + name + '.csv', index=False)
        # test.to_csv('csv/' + name + '-train.csv', index=False)
        # test.to_csv('csv/' + name + '-test.csv', index=False)

        datetime_columns = ['index', 'name']
        target_column = 'value'

        feature_columns = datetime_columns + ['value']

        resample_df = train[feature_columns]
        resample_df_test = test[feature_columns]
        resample_df_val=val[feature_columns]
        #print(resample_df)

        plot_length = 1000000
        plot_df = resample_df.copy(deep=True).iloc[:plot_length]
        #plot_df['weekday'] = plot_df['date'].dt.day_name()

        n_train = len(train_np)
        n_test = len(test_np)

        features = ['index', 'value']
        feature_array = resample_df[features].values
        feature_array_test = resample_df_test[features].values
        feature_array_val = resample_df_val[features].values
        # Fit Scaler only on Training features
        feature_scaler = MinMaxScaler()
        feature_scaler.fit(feature_array[:n_train])
        # Fit Scaler only on Training target values
        #feature_scaler.fit(feature_array_test[:n_test])

        target_scaler = MinMaxScaler()
        target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

        # Transfom on both Training and Test data
        scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                                    columns=features)

        scaled_array_test = pd.DataFrame(feature_scaler.transform(feature_array_test),
                                         columns=features)
        scaled_array_val = pd.DataFrame(feature_scaler.transform(feature_array_val),
                                         columns=features)

        sequence_length = 10

        X_train, y_train = create_sliding_window(scaled_array,
                                                 sequence_length)

        X_test, y_test = create_sliding_window(scaled_array_test,
                                               sequence_length)
        X_val, y_val = create_sliding_window(scaled_array_val,
                                               sequence_length)

        #sns.displot(data=y_test,  kind="hist", bins=100, aspect=1.5)
      #  plt.show()
       # pdb.set_trace()

        #f = Fitter(y_train,
         #          distributions=['gamma',
          #                        'lognorm',
           #                       "beta",
            #                      "burr",
             #                     "norm",'cauchy', 'chi2', 'expon', 'exponpow', 'powerlaw', 'rayleigh', 'uniform'])
        #f.fit()
        #f.summary()
        #pdb.set_trace()
        n_features = scaled_array.shape[-1]
        sequence_length = 10
        output_length = 1

        batch_size = 64
        n_epochs = 10
        learning_rate = 0.01

        bayesian_lstm = MCD_AE(n_features=n_features,
                                     output_length=output_length,
                                     batch_size=batch_size)
        model_parameters = filter(lambda p: p.requires_grad, bayesian_lstm.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        print("total_parameters", params)

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
        bayesian_lstm.train()
        start=time.time()

        for e in range(1, n_epochs + 1):
            for b in range(0, len(X_train), batch_size):
                features = X_train[b:b + batch_size, :, :]
                target = y_train[b:b + batch_size]

                X_batch = torch.tensor(features, dtype=torch.float32)
                y_batch = torch.tensor(target, dtype=torch.float32)
                # pdb.set_trace()
              
               

                output = bayesian_lstm(X_batch)
                loss = criterion(output.view(-1), y_batch)


                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if e % 10 == 0:
                print('epoch', e, 'loss: ', loss.item())
                offset = sequence_length
        end=time.time()        
        T.append((end-start)/n_epochs)
        print('Training_time',(end-start)/n_epochs,'signal', name)


        training_df = pd.DataFrame()
       # training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1]
        training_df['index'] = resample_df['index'].iloc[offset:n_train + offset:1]
        training_predictions = bayesian_lstm.predict(X_train)


        training_df['value'] = inverse_transform(training_predictions)
        training_df['source'] = 'Training Prediction'


        #training_truth_df['date'] = training_df['date']
        training_truth_df['index'] = training_df['index']
        training_truth_df['value'] = resample_df['value'].iloc[
                                     offset:n_train + offset:1]
        training_truth_df['source'] = 'True Values'

        testing_df = pd.DataFrame()
        #testing_df['date'] = resample_df_test['date'].iloc[offset:n_test + offset:1]
        testing_df['index'] = resample_df_test['index'].iloc[offset:n_test + offset:1]
        start=time.time()

        testing_predictions = bayesian_lstm.predict(X_test)
        end=time.time()
        T2.append((end-start))
        print('Test_time',(end-start),'signal', name)

        testing_df['value'] = inverse_transform(testing_predictions)
        testing_df['source'] = 'Test Prediction'

        testing_truth_df = pd.DataFrame()
        #testing_truth_df['date'] = testing_df['date']
        testing_truth_df['index'] = testing_df['index']

        testing_truth_df['value'] = resample_df_test['value'].iloc[offset:n_test + offset:1]
        testing_truth_df['source'] = 'True Values'


        evaluation = pd.concat([training_df,
                                testing_df,
                                training_truth_df,
                                testing_truth_df
                                ], axis=0)

        fig = px.line(evaluation,
                             x="index",
                              y="value",
                              color="source")
        #fig.show()

        n_experiments = 2
        test_uncertainty_df = pd.DataFrame()
        #test_uncertainty_df['date'] = testing_df['date']
        test_uncertainty_df['index'] = testing_df['index']

        for i in range(n_experiments):
            experiment_predictions = bayesian_lstm.predict(X_test)

            test_uncertainty_df['value_{}'.format(i)] = inverse_transform(experiment_predictions)

        log_energy_consumption_df = test_uncertainty_df.filter(like='value', axis=1)
        test_uncertainty_df['value_mean'] = log_energy_consumption_df.mean(axis=1)
        test_uncertainty_df['value_std'] = log_energy_consumption_df.std(axis=1)


        test_uncertainty1 = test_uncertainty_df['value_mean']

        #########################Adaptive Smoothing

        #   pdb.set_trace()
        Threshold = 10
        gamma = 0.1
        betta = 0.1
        test_uncertainty2 = []
        ###########
        for jj in range(len(test_uncertainty1) - 2):
            test_uncertaintyy = (1 / (1 + gamma + betta)) * (
                        test_uncertainty1.iloc[jj] + gamma * test_uncertainty1.iloc[jj + 1] + betta *
                        test_uncertainty1.iloc[jj + 2])
            test_uncertainty2.append(test_uncertaintyy)
        test_uncertainty2 = pd.DataFrame(test_uncertainty2)

        test_uncertainty2.columns = ["value_mean"]


        test_uncertainty_df["value_mean"] = test_uncertainty2["value_mean"]

        ########################################################


        test_uncertainty_df = test_uncertainty_df[['index',  'value_mean', 'value_std']]
        test_uncertainty_df['lower_bound'] = test_uncertainty_df['value_mean'] - 3* test_uncertainty_df['value_std']
        test_uncertainty_df['upper_bound'] = test_uncertainty_df['value_mean'] + 3 * test_uncertainty_df['value_std']
        import plotly.graph_objects as go

        test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
        # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
        truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
        # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

        upper_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['upper_bound'],
          mode='lines',
         fill=None,
         name='99% Upper Confidence Bound'
         )
        lower_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['lower_bound'],
          mode='lines',
          fill='tonexty',
         name= '99% Lower Confidence Bound',
         fillcolor='rgba(255, 211, 0, 0.1)',
         )
        real_trace = go.Scatter(
            x=truth_uncertainty_plot_df['index'],
            y=truth_uncertainty_plot_df['value'],
            mode='lines',
            fill=None,
            name='Real Values'
        )

        labels = go.Scatter(
           x=label_data['index'],
          y=label_data['value'],
         mode='lines',
         fill='tonexty' ,
         name='labels'
         )
        data = [upper_trace, lower_trace, real_trace]

        fig = go.Figure(data=data)
        fig.update_layout(title='Uncertainty MCDropout Test Data',
                            xaxis_title='index',
                            yaxis_title='value',
         legend_font_size=14,
                          )
        #fig.show()
        bounds_df = pd.DataFrame()

        # Using 99% confidence bounds
        bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
        bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
        bounds_df['real_value'] = truth_uncertainty_plot_df['value']
        bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

        bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                                  (bounds_df['real_value'] <= bounds_df['upper_bound']))

        print("Proportion of points contained within 99% confidence interval:",
              bounds_df['contained'].mean())
        predictedanomaly = bounds_df.index[~bounds_df['contained']]






        realanomaly = label_data['index']

        predicter=list(range(len(test_uncertainty2)))

        a1 = pd.DataFrame(index=range(len(test_uncertainty2)),columns=range(2))
        a1.columns=['index','value']

        a2 = pd.DataFrame(index=range(len(test_uncertainty2)), columns=range(2))
        a2.columns = ['index','value']




        for i in range(len(predicter)):
            if i in predictedanomaly:
                a1.iloc[i,1]=1
            else:
                a1.iloc[i,1]=0

        for i in range(len(predicter)):
            if i in realanomaly:
                a2.iloc[i,1] = 1
            else:
                a2.iloc[i, 1] = 0

        y_real=a2.value
        y_real=y_real.astype(int)
        y_predi=a1.value
        y_predi=y_predi.astype(int)


        cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
        cm_plot_labels = ['no_anomaly', 'had_anomaly']
        plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

        # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
        #fp = len(predicteddanomaly) - tp
        #fn = 0
        #tn = len(truth_uncertainty_plot_df) - tp - fp - fn

        tp=cm[0][0]
        fp=cm[0][1]
        fn=cm[1][0]
        tn=cm[1][1]

        precision1 = tp / (tp + fp)
        recall1 = tp / (tp + fn)
        Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
        F11 = 2 / ((1 / precision1) + (1 / recall1))
      
        precision.append(precision1)
        F1.append(F11)
        Accuracy.append(Accuracy1)
        recall.append(recall1)

        im=im+1

        #matched_indices = list(i_anom_predicted & true_indices_flat)


recall_final=mean(recall)
precision_final=mean(precision)
F1_final=mean(F1)
Accuracy_final=mean(Accuracy)
#cm = confusion_matrix(y_true=test_labels, y_pred=predicteddanomaly)

################################################################################

# %%

## New Test Bayesian LSTM AE



total_parameters 545217
epoch 10 loss:  0.045960117131471634
Training_time 5.097353935241699 signal P-1
Test_time 5.503154754638672 signal P-1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:493: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.04755738669805768
total_parameters 545217
epoch 10 loss:  0.11617942154407501
Training_time 4.097381281852722 signal S-1
Test_time 4.574758768081665 signal S-1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:493: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.002458680508127305
total_parameters 545217
epoch 10 loss:  0.26880523562431335
Training_time 4.227539539337158 signal E-1
Test_time 4.750142574310303 signal E-1


...

Proportion of points contained within 99% confidence interval: 0.5000597728631201
total_parameters 545217
epoch 10 loss:  0.15432997047901154
Training_time 1.1012699365615846 signal A-9
Test_time 4.914627552032471 signal A-9


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:493: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.2060778727445394
total_parameters 545217
epoch 10 loss:  0.11983682215213776
Training_time 4.100517201423645 signal F-3
Test_time 4.86653733253479 signal F-3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:493: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.3582357159933062


In [ ]:
import io
import os
import urllib.request
import zipfile
import datetime
import pdb
import holidays
import json
import copy
import numpy as np
import torch
import random
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import seaborn as sns
from datetime import datetime
import pandas as pd
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'
from statistics import mean
from sklearn.metrics import confusion_matrix
import itertools
seed=42
torch.backends.cudnn.determinestic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())

    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()
#from data import load_signal
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
import time
train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
       
  
def build_df(data, start=0):
#
    index = np.array(range(start, start + len(data)))
    timestamp = index * 86400 + 1022819200



    return pd.DataFrame({'timestamp': timestamp.astype(int), 'value': data[:, 0], 'index': index.astype(int)})


def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)


def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))
class MCD_AE(nn.Module):

    def __init__(self, n_features, output_length, batch_size):
        super(MCD_AE, self).__init__()

        self.batch_size = batch_size  # user-defined

        self.hidden_size_1 = 128
        self.hidden_size_2 = 64
        self.hidden_size_3= 16
        self.hidden_size_4 = 64
        self.stacked_layers = 3
        self.dropout_probability = 0.5  # arbitrary value (the paper suggests that performance is generally stable across all ranges)


        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm3 = nn.LSTM(self.hidden_size_2,
                             self.hidden_size_3,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm4 = nn.LSTM(self.hidden_size_3,
                             self.hidden_size_4,
                             num_layers=self.stacked_layers,
                             batch_first=True)

        self.fc = nn.Linear(self.hidden_size_4, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)

        output, _ = self.lstm1(x, hidden)

        #output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        state = self.init_hidden3(batch_size)
        output, state = self.lstm3(output, state)
        state = self.init_hidden4(batch_size)
        output, state = self.lstm4(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :]  # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state
    def init_hidden3(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_3))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_3))
        return hidden_state, cell_state
    def init_hidden4(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_4))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_4))
        return hidden_state, cell_state
    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()
###Prepare Labels
CSV_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

# %%
os.makedirs('csv', exist_ok=True)

# %%
selected_columns = ['index', 'name',  'value']



df_label = pd.read_csv(CSV_URL)

#name='F-7'
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']






SMAP=pd.Series(SMAP)
MSL=pd.Series(MSL)

#pdb.set_trace()
avg=[]
train_signals=MSL
precision=[]
recall=[]
Accuracy=[]
F1=[]
im=0
training_truth_df=pd.DataFrame()
T=[]
T2=[]
for name in MSL:

        label_row = df_label[df_label.chan_id == name]
    # pdb.set_trace()

      #  labels = label_row.anomaly_sequences[im]
       # labels = eval(labels)

        #true_indices_grouped = [list(range(e[0], e[1] + 1)) for e in labels]
        #true_indices_flat = set([i for group in true_indices_grouped for i in group])

        appended_data = []

        #labels = true_indices_flat

    # for i in range(len(labels)):
    #    anom=labels[i]
    # start=anom[0]
    # end=anom[1]

        #index = list(labels)

    # timestamp = index * 86400 + 1022819200

        #anomalies = pd.DataFrame({'value': 1, 'index': index})
    #     appended_data.append(anomalies)

        #label_data = anomalies

    # label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
    # label_data ['month'] = label_data ['date'].dt.month.astype(int)
        #label_data['name'] = name
    # label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
    # label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)


        #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
        #label_data ['month'] = label_data ['date'].dt.month.astype(int)
        #label_data ['name'] = name
        #label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
        #label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
        #label_data  = label_data [selected_columns]
        #label_data .to_csv('csv/' + name + '.csv', index=False)


        signal=name
        train_np = np.load('data/train/' + signal + '.npy')
        test_np = np.load('data/test/' + signal + '.npy')


        data = build_df(np.concatenate([train_np, test_np]))
        data['date'] = pd.to_datetime(data['timestamp'], unit='s')
        data['month'] = data['date'].dt.month.astype(int)
        data['name'] = name
        data['index'] = data['index'].astype(int)
        data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
        data['hour_of_day'] = data['date'].dt.hour.astype(int)
        data = data[selected_columns]
        _ , validation = train_test_split(
            data,
            test_size=0.15
        )

        val_dataset=validation

        val=val_dataset


        #val['index'] = val['index'].astype(int)
        #val['date'] = pd.to_datetime(val['timestamp'], unit='s')
        #val['month'] = val['date'].dt.month.astype(int)
        #val['day_of_month'] = val['date'].dt.day.astype(int)
        #val['name'] = name
        #val['day_of_week'] = val['date'].dt.dayofweek.astype(int)
        #val['hour_of_day'] = val['date'].dt.hour.astype(int)
        #val['index'] = val['index'].astype(int)
        #val = val[selected_columns]

        train = build_df(train_np)
        train['date'] = pd.to_datetime(train['timestamp'], unit='s')
        train['month'] = train['date'].dt.month.astype(int)
        train['day_of_month'] = train['date'].dt.day.astype(int)
        train['name'] = name
        train['day_of_week'] = train['date'].dt.dayofweek.astype(int)
        train['hour_of_day'] = train['date'].dt.hour.astype(int)
        train['index'] = train['index'].astype(int)
        train = train[selected_columns]
        # train.to_csv('csv/' + name + '.csv', index=False)
        # train.to_csv('csv/' + name + '-train.csv', index=False)


        test = build_df(test_np, start=len(train))
        test['date'] = pd.to_datetime(test['timestamp'], unit='s')
        test['month'] = test['date'].dt.month.astype(int)
        test['name'] = name
        test['day_of_week'] = test['date'].dt.dayofweek.astype(int)
        test['hour_of_day'] = test['date'].dt.hour.astype(int)
        test['index'] = test['index'].astype(int)
        test = test[selected_columns]
        # test.to_csv('csv/' + name + '.csv', index=False)
        # test.to_csv('csv/' + name + '-train.csv', index=False)
        # test.to_csv('csv/' + name + '-test.csv', index=False)

        datetime_columns = ['index', 'name']
        target_column = 'value'

        feature_columns = datetime_columns + ['value']

        resample_df = train[feature_columns]
        resample_df_test = test[feature_columns]
        resample_df_val=val[feature_columns]
        #print(resample_df)

        plot_length = 1000000
        plot_df = resample_df.copy(deep=True).iloc[:plot_length]
        #plot_df['weekday'] = plot_df['date'].dt.day_name()

        n_train = len(train_np)
        n_test = len(test_np)

        features = ['index', 'value']
        feature_array = resample_df[features].values
        feature_array_test = resample_df_test[features].values
        feature_array_val = resample_df_val[features].values
        # Fit Scaler only on Training features
        feature_scaler = MinMaxScaler()
        feature_scaler.fit(feature_array[:n_train])
        # Fit Scaler only on Training target values
        #feature_scaler.fit(feature_array_test[:n_test])

        target_scaler = MinMaxScaler()
        target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

        # Transfom on both Training and Test data
        scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                                    columns=features)

        scaled_array_test = pd.DataFrame(feature_scaler.transform(feature_array_test),
                                         columns=features)
        scaled_array_val = pd.DataFrame(feature_scaler.transform(feature_array_val),
                                         columns=features)

        sequence_length = 25

        X_train, y_train = create_sliding_window(scaled_array,
                                                 sequence_length)

        X_test, y_test = create_sliding_window(scaled_array_test,
                                               sequence_length)
        X_val, y_val = create_sliding_window(scaled_array_val,
                                               sequence_length)

        #sns.displot(data=y_test,  kind="hist", bins=100, aspect=1.5)
      #  plt.show()
       # pdb.set_trace()

        #f = Fitter(y_train,
         #          distributions=['gamma',
          #                        'lognorm',
           #                       "beta",
            #                      "burr",
             #                     "norm",'cauchy', 'chi2', 'expon', 'exponpow', 'powerlaw', 'rayleigh', 'uniform'])
        #f.fit()
        #f.summary()
        #pdb.set_trace()
        n_features = scaled_array.shape[-1]
        sequence_length = 25
        output_length = 1

        batch_size = 64
        n_epochs = 10
        learning_rate = 0.01

        bayesian_lstm = MCD_AE(n_features=n_features,
                                     output_length=output_length,
                                     batch_size=batch_size)
        model_parameters = filter(lambda p: p.requires_grad, bayesian_lstm.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        print("total_parameters", params)

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
        bayesian_lstm.train()
        start=time.time()

        for e in range(1, n_epochs + 1):
            for b in range(0, len(X_train), batch_size):
                features = X_train[b:b + batch_size, :, :]
                target = y_train[b:b + batch_size]

                X_batch = torch.tensor(features, dtype=torch.float32)
                y_batch = torch.tensor(target, dtype=torch.float32)
                # pdb.set_trace()
              
               

                output = bayesian_lstm(X_batch)
                loss = criterion(output.view(-1), y_batch)


                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if e % 10 == 0:
                print('epoch', e, 'loss: ', loss.item())
                offset = sequence_length
        end=time.time()        
        T.append((end-start)/n_epochs)
        print('Training_time',(end-start)/n_epochs,'signal', name)


        training_df = pd.DataFrame()
       # training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1]
        training_df['index'] = resample_df['index'].iloc[offset:n_train + offset:1]
        training_predictions = bayesian_lstm.predict(X_train)


        training_df['value'] = inverse_transform(training_predictions)
        training_df['source'] = 'Training Prediction'


        #training_truth_df['date'] = training_df['date']
        training_truth_df['index'] = training_df['index']
        training_truth_df['value'] = resample_df['value'].iloc[
                                     offset:n_train + offset:1]
        training_truth_df['source'] = 'True Values'

        testing_df = pd.DataFrame()
        #testing_df['date'] = resample_df_test['date'].iloc[offset:n_test + offset:1]
        testing_df['index'] = resample_df_test['index'].iloc[offset:n_test + offset:1]
        start=time.time()

        testing_predictions = bayesian_lstm.predict(X_test)
        end=time.time()
        T2.append((end-start))
        print('Test_time',(end-start),'signal', name)

        testing_df['value'] = inverse_transform(testing_predictions)
        testing_df['source'] = 'Test Prediction'

        testing_truth_df = pd.DataFrame()
        #testing_truth_df['date'] = testing_df['date']
        testing_truth_df['index'] = testing_df['index']

        testing_truth_df['value'] = resample_df_test['value'].iloc[offset:n_test + offset:1]
        testing_truth_df['source'] = 'True Values'


        evaluation = pd.concat([training_df,
                                testing_df,
                                training_truth_df,
                                testing_truth_df
                                ], axis=0)

        fig = px.line(evaluation,
                             x="index",
                              y="value",
                              color="source")
        #fig.show()

        n_experiments = 2
        test_uncertainty_df = pd.DataFrame()
        #test_uncertainty_df['date'] = testing_df['date']
        test_uncertainty_df['index'] = testing_df['index']

        for i in range(n_experiments):
            experiment_predictions = bayesian_lstm.predict(X_test)

            test_uncertainty_df['value_{}'.format(i)] = inverse_transform(experiment_predictions)

        log_energy_consumption_df = test_uncertainty_df.filter(like='value', axis=1)
        test_uncertainty_df['value_mean'] = log_energy_consumption_df.mean(axis=1)
        test_uncertainty_df['value_std'] = log_energy_consumption_df.std(axis=1)


        test_uncertainty1 = test_uncertainty_df['value_mean']

        #########################Adaptive Smoothing

        #   pdb.set_trace()
        Threshold = 10
        gamma = 0.1
        betta = 0.1
        test_uncertainty2 = []
        ###########
        for jj in range(len(test_uncertainty1) - 2):
            test_uncertaintyy = (1 / (1 + gamma + betta)) * (
                        test_uncertainty1.iloc[jj] + gamma * test_uncertainty1.iloc[jj + 1] + betta *
                        test_uncertainty1.iloc[jj + 2])
            test_uncertainty2.append(test_uncertaintyy)
        test_uncertainty2 = pd.DataFrame(test_uncertainty2)

        test_uncertainty2.columns = ["value_mean"]


        test_uncertainty_df["value_mean"] = test_uncertainty2["value_mean"]

        ########################################################


        test_uncertainty_df = test_uncertainty_df[['index',  'value_mean', 'value_std']]
        test_uncertainty_df['lower_bound'] = test_uncertainty_df['value_mean'] - 3* test_uncertainty_df['value_std']
        test_uncertainty_df['upper_bound'] = test_uncertainty_df['value_mean'] + 3 * test_uncertainty_df['value_std']
        import plotly.graph_objects as go

        test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
        # test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
        truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
        # truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

        upper_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['upper_bound'],
          mode='lines',
         fill=None,
         name='99% Upper Confidence Bound'
         )
        lower_trace = go.Scatter(
            x=test_uncertainty_plot_df['index'],
           y=test_uncertainty_plot_df['lower_bound'],
          mode='lines',
          fill='tonexty',
         name= '99% Lower Confidence Bound',
         fillcolor='rgba(255, 211, 0, 0.1)',
         )
        real_trace = go.Scatter(
            x=truth_uncertainty_plot_df['index'],
            y=truth_uncertainty_plot_df['value'],
            mode='lines',
            fill=None,
            name='Real Values'
        )

       # labels = go.Scatter(
        #   x=label_data['index'],
         # y=label_data['value'],
         #mode='lines',
         #fill='tonexty' ,
         #name='labels'
         #)
        data = [upper_trace, lower_trace, real_trace]

        fig = go.Figure(data=data)
        fig.update_layout(title='Uncertainty MCDropout Test Data',
                            xaxis_title='index',
                            yaxis_title='value',
         legend_font_size=14,
                          )
        #fig.show()
        bounds_df = pd.DataFrame()

        # Using 99% confidence bounds
        bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
        bounds_df['prediction'] = test_uncertainty_plot_df['value_mean']
        bounds_df['real_value'] = truth_uncertainty_plot_df['value']
        bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

        bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                                  (bounds_df['real_value'] <= bounds_df['upper_bound']))

        print("Proportion of points contained within 99% confidence interval:",
              bounds_df['contained'].mean())
        predictedanomaly = bounds_df.index[~bounds_df['contained']]






       # realanomaly = label_data['index']

        #predicter=list(range(len(test_uncertainty2)))

        a1 = pd.DataFrame(index=range(len(test_uncertainty2)),columns=range(2))
        a1.columns=['index','value']

        a2 = pd.DataFrame(index=range(len(test_uncertainty2)), columns=range(2))
        a2.columns = ['index','value']




#        for i in range(len(predicter)):
 #           if i in predictedanomaly:
  #              a1.iloc[i,1]=1
   #         else:
    #            a1.iloc[i,1]=0

        #for i in range(len(predicter)):
         #   if i in realanomaly:
          #      a2.iloc[i,1] = 1
           # else:
            #    a2.iloc[i, 1] = 0

        #y_real=a2.value
       # y_real=y_real.astype(int)
       # y_predi=a1.value
       # y_predi=y_predi.astype(int)


        #cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
        #cm_plot_labels = ['no_anomaly', 'had_anomaly']
        #plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

        # tp = len([np.where(predicteddanomaly == x)[0] for x in realanomaly])
        #fp = len(predicteddanomaly) - tp
        #fn = 0
        #tn = len(truth_uncertainty_plot_df) - tp - fp - fn

        #tp=cm[0][0]
        #fp=cm[0][1]
        #fn=cm[1][0]
        #tn=cm[1][1]

        #precision1 = tp / (tp + fp)
        #recall1 = tp / (tp + fn)
        #Accuracy1 = (tp + tn) / len(truth_uncertainty_plot_df)
        #F11 = 2 / ((1 / precision1) + (1 / recall1))
      
        #precision.append(precision1)
        #F1.append(F11)
        #Accuracy.append(Accuracy1)
        #recall.append(recall1)

        #im=im+1

        #matched_indices = list(i_anom_predicted & true_indices_flat)


#recall_final=mean(recall)
#precision_final=mean(precision)
#F1_final=mean(F1)
#Accuracy_final=mean(Accuracy)
#cm = confusion_matrix(y_true=test_labels, y_pred=predicteddanomaly)

################################################################################

# %%

## New Test Bayesian LSTM AE



total_parameters 545217
epoch 10 loss:  1.4802489545218123e-07
Training_time 6.244893336296082 signal M-6
Test_time 2.8344850540161133 signal M-6


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:493: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.41650197628458496
total_parameters 545217
epoch 10 loss:  0.0447407141327858
Training_time 7.323600721359253 signal M-1
Test_time 2.8106515407562256 signal M-1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:493: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.010657193605683837
total_parameters 545217
epoch 10 loss:  0.002773695858195424
Training_time 7.324299049377442 signal M-2
Test_time 2.7761430740356445 signal M-2


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:493: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.0641735918744229
total_parameters 545217
epoch 10 loss:  3.087450750172138e-05
Training_time 5.276236510276794 signal M-7
Test_time 2.504493236541748 signal M-7


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:493: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Proportion of points contained within 99% confidence interval: 0.08493664946034725
total_parameters 545217
epoch 10 loss:  0.10591845214366913
Training_time 11.260765790939331 signal F-8
Test_time 2.852597236633301 signal F-8
Proportion of points contained within 99% confidence interval: 0.015028432168968318


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:493: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:

df_MSL=pd.DataFrame()
df_MSL['training_time']=T
df_MSL['testing_time']=T2
df_MSL['pipline']='MCD'
df_MSL.to_csv('df_msl.csv',index=False)